In [10]:
from USTs import USTs
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from scipy.optimize import newton
import pandas_market_calendars as mcal
import pandas as pd
import numpy as np

In [11]:
# Bill pricing
issue_date = datetime.date(2025, 2, 4)
maturity_date = datetime.date(2025, 4, 24)

bill_1 = USTs(auction_data= None, price_data=None).get_bill_discount_rate(price=99.0808021,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_1
issue_date = datetime.date(2025, 2, 4)
maturity_date = datetime.date(2025, 4, 24)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=99.0808021,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2
issue_date = datetime.date(1990, 6, 7)
maturity_date = datetime.date(1991, 6, 6)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=92.265000,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

8.237

In [12]:
holiday_array = mcal.get_calendar('NYSE').holidays().holidays
start, end = datetime.date(1990, 1, 1), datetime.date(2060, 1, 1)
holidays_list = [pd.to_datetime(np_date).date() for np_date in holiday_array]
holidays = [holiday for holiday in holidays_list if start < holiday < end]
nyse_calendar = mcal.get_calendar('NYSE').regular_holidays

def adjust_for_bad_day(date: datetime.date) -> datetime.date: # type: ignore
    while date.weekday() in [5, 6] or date in holidays:
        date = date + timedelta(days=1)
    return date

def get_dates_and_cashflows(issue_date: datetime.date,
                            maturity_date: datetime.date,
                            coupon: float,
                            FV: int = 100):
    payment_dates = [maturity_date]
    current_date = maturity_date
    while current_date > issue_date:
        current_date = maturity_date - relativedelta(months=len(payment_dates) * 6)
        if current_date > issue_date:
            payment_dates.append(current_date)
    payment_dates = sorted(payment_dates)
    
    coupon_amt = coupon / 2
    return_list = list()
    for date in payment_dates[:-1]:
        return_list.append((date, coupon_amt))
    return_list.append((payment_dates[-1], coupon_amt + FV))
    return return_list

def calculate_accrued_interest(dates_and_cashflows, as_of_date, issue_date) -> float:
    for date, _ in dates_and_cashflows:
        if date > as_of_date:
            next_coupon_date = date
            previous_index = dates_and_cashflows.index((date, _)) - 1
            if previous_index >= 0:
                last_coupon_date = dates_and_cashflows[previous_index][0]
            else:
                last_coupon_date = issue_date
            break

    days_in_period = (next_coupon_date - (next_coupon_date - relativedelta(months=6))).days
    days_accrued = (as_of_date - last_coupon_date).days
    if days_in_period == 0:
        return 0.0

    coupon_amt = dates_and_cashflows[0][1]
    accrued = coupon_amt * days_accrued / days_in_period
    return accrued

def calculate_bond_price(issue_date: datetime.date,
                         maturity_date: datetime.date,
                         as_of_date: datetime.date,
                         coupon: float,
                         discount_rate: float,
                         dirty: bool = False) -> float:
    
    dates_and_cashflows = get_dates_and_cashflows(issue_date=issue_date,
                                                      maturity_date=maturity_date,
                                                      coupon=coupon)
    for unadjusted_date, _ in dates_and_cashflows:
        date = adjust_for_bad_day(unadjusted_date)
        if date > as_of_date:
            time_to_coupon = (date - as_of_date).days
            coupon_period = (date - (date - relativedelta(months=6))).days
            break
    period_fraction = time_to_coupon / coupon_period

    PV = 0
    full_periods = 0
    DISCOUNT_RATE = discount_rate
    for date, cashflow in dates_and_cashflows:
        if adjust_for_bad_day(date) > as_of_date:
            exponent = period_fraction + full_periods
            pmt_value = cashflow/((1 + (DISCOUNT_RATE/2))**(exponent))
            full_periods += 1
            PV += pmt_value
    
    if not dirty:
        accrued = calculate_accrued_interest(dates_and_cashflows=dates_and_cashflows,
                                             as_of_date=as_of_date,
                                             issue_date=issue_date)
        print(f"Accrued: {accrued}")
        PV -= accrued
    print(full_periods)
    return PV

def error_function(discount_rate: float,
                   price: float,
                   issue_date: datetime.date,
                   maturity_date: datetime.date,
                   as_of_date: datetime.date,
                   coupon: float,
                   dirty: bool=False):
    calculated_price = calculate_bond_price(issue_date=issue_date,
                                            maturity_date=maturity_date,
                                            discount_rate=discount_rate,
                                            coupon=coupon,
                                            as_of_date=as_of_date,
                                            dirty=dirty)
    error = price - calculated_price
    return error


def get_coupon_ytm(price: float, 
            issue_date: datetime.date,
            maturity_date: datetime.date,
            as_of_date: datetime.date,
            coupon: float,
            dirty: bool=False):
    guess = coupon / 100

    try:
        ytm = newton(
            func=error_function,
            x0=guess,
            args=(price, issue_date, maturity_date, as_of_date, coupon, dirty)
        )
        return round(float(ytm * 100), 6)
    except RuntimeError:
        return None

In [19]:
PRICE = 100.3267
ISSUE = datetime.datetime(2001, 8, 31)
MATURITY = datetime.datetime(2003, 8, 31)
RATE = 3.625
SETTLEMENT = datetime.datetime(2001, 9, 11)

In [20]:
get_coupon_ytm(price=PRICE,
        issue_date=ISSUE.date(),
        maturity_date=MATURITY.date(),
        as_of_date=SETTLEMENT.date(),
        coupon=RATE)

Accrued: 0.10835597826086957
4
Accrued: 0.10835597826086957
4
Accrued: 0.10835597826086957
4
Accrued: 0.10835597826086957
4


3.466447

In [21]:
import rateslib as rl
bond = rl.FixedRateBond(
    effective=ISSUE,
    termination=MATURITY,
    fixed_rate=RATE,
    spec="us_gb"  # US Government Bond
)
bond.ytm(price=PRICE, settlement=SETTLEMENT, dirty=False)

3.4515337604883825

In [22]:
bond.accrued(settlement=SETTLEMENT)

0.11015193370165746

In [23]:
USTs(auction_data=None,price_data=None).get_coupon_ytm(price=PRICE, issue_date=ISSUE.date(),
                    maturity_date=MATURITY.date(),
                    as_of_date=SETTLEMENT.date(),
                    coupon=RATE)

0.11015193370165746
0.11015193370165746
0.11015193370165746
0.11015193370165746
0.11015193370165746
0.11015193370165746


3.453464

In [24]:
(datetime.date(2001, 7, 31) - datetime.date(2001, 1, 31)).days

181